In [7]:
# Troubleshooting for calculate_state in multisensory_playbook 
import numpy as np
import pandas as pd
from numpy.random import choice

In [4]:
repeats = 1
windowsize = 4
A = choice([-1, 0, 1], size=5000, p=[0.33,0.34,0.33]) #np.array([1,1,1,0,0,1,1,1,0,0,0])
V = choice([-1, 0, 1], size=5000, p=[0.33,0.34,0.33]) #np.array([1,1,1,0,0,1,1,1,0,0,0])

In [5]:
def calculate_state( draw_sequence):
    # Mapping for the states to digits
    state_to_digit = {-1: 0, 0: 1, 1: 2} # To change -1 to 1

    # Convert the draw sequence to a base-3 number
    base_3_number = 0
    for draw in draw_sequence:
        base_3_number = base_3_number * 3 + state_to_digit[draw]
    
    # The state is the base-3 number
    return base_3_number

def apply_state( row):
    # Convert row to list and pass it to the calculate_state function
    return calculate_state(row.tolist())

max_state = 3**(2*windowsize) # 3**(2n) 

C = np.zeros((repeats, max_state))
for trialnum in range(repeats):
    _A = A#[trialnum]
    _V = V#[trialnum]
    df = pd.DataFrame()
    df['A'], df['V'] = _A, _V
    
if windowsize == 2:
    df['A-1'], df['V-1'] =  df['A'].shift(1), df['V'].shift(1) # Shifting column down one step
if windowsize == 3:
    df['A-1'], df['V-1'] =  df['A'].shift(1), df['V'].shift(1) # Shifting column down one step
    df['A-2'], df['V-2'] =  df['A'].shift(2), df['V'].shift(2) # Shifting column down one step (window size is 3)
if windowsize == 4:
    df['A-1'], df['V-1'] =  df['A'].shift(1), df['V'].shift(1) # Shifting column down one step
    df['A-2'], df['V-2'] =  df['A'].shift(2), df['V'].shift(2) # Shifting column down one step (window size is 3)
    df['A-3'], df['V-3'] =  df['A'].shift(3), df['V'].shift(3) # Shifting column down one step (window size is 4)

df = df.dropna()

# Apply the function to each row and store the result in a new column 'state'
df['state'] = df.apply(apply_state, axis=1)
# Calculate value counts
state_counts = df['state'].value_counts()


# Generate a range of numbers representing all possible states
# Adjust the range based on your specific needs (max_state + 1)
all_possible_states = range(0, max_state)  # Replace max_state with your actual maximum state value

# Reindex the value counts to include all possible states
# Fill missing values (states with 0 occurrences) with 0
state_counts = state_counts.reindex(all_possible_states, fill_value=0)
C[trialnum,:] = state_counts


In [6]:
df.describe()


6561

In [127]:
# Verify if unique states correspond to unique rows
column_name = 'state'

# Get unique values in the specified column
unique_values = df[column_name].unique()

# Iterate over unique values
for value in unique_values:
    # Filter DataFrame for rows where the specified column has the current unique value
    matching_rows = df[df[column_name] == value]
    
    # Reset index to exclude the index column
    matching_rows.reset_index(drop=True, inplace=True)
    
    # Check if all rows are identical
    if matching_rows.equals(pd.concat([matching_rows.iloc[[0]]] * len(matching_rows), ignore_index=True)):
        #print(f"All rows with {column_name} = {value} are identical:")
        #print(matching_rows)
        s=2 # placeholder to pass if statement
    else:
        print(f"Rows with {column_name} = {value} have different values:")
        print(matching_rows)

In [ ]:
0 6
1 3**2
2 3**4
3 3**6
4 3**8